<a href="https://colab.research.google.com/github/elsonyeh/computer-programming-and-application/blob/main/EX05_02_Ask_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##可以跟 Gemini 聊天的 Line Bot

In [1]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 6.6 MB/s eta 0:00:00


In [6]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
  ngrok.set_auth_token(userdata.get('ngrok_token'))
  ngrok.connect(5000)
  run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import (
  WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

LINE_CHANNEL_ACCESS_TOKEN = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
LINE_CHANNEL_SECRET = userdata.get('LINE_CHANNEL_SECRET')
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-2.0-flash')
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        response = ask_gemini(event.message.text)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=response)]
            )
        )


# 啟動 pyngrok + Flask
if __name__ == "__main__":
    # 設定 ngrok token
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # 開 ngrok 隧道，監聽 5000 port
    public_url = ngrok.connect(5000).public_url
    print("👉 請把這個網址填到 LINE Developer Webhook URL：")
    print(f"{public_url}/callback")

    # 啟動 Flask Server
    app.run()

👉 請把這個網址填到 LINE Developer Webhook URL：
https://a6df-34-13-133-24.ngrok-free.app/callback
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Jun/2025 23:01:23] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jun/2025 23:01:28] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jun/2025 23:02:17] "POST /callback HTTP/1.1" 200 -
